# How a tax funded UBI can improve the lives of Indigenous Americans
*By Nate Golden, 2020-10-12*

Indigenous People’s Day is a time to celebrate and honor Indigenous Americans while also recognizing the history of discrimination, oppression, and genocide they have faced since Europeans arrived in the Americas more than 500 years ago.

For centuries American public policy has treated Indigenous Americans as second class citizens—enforcing migration, barring citizenship, and refusing to give them the fundamental rights granted to them by the Constitution.  All of this produced large economic disparities that continue to this day.  Indigenous Americans are 35 percent more likely to be in poverty than non-Indigenous Americans, and their median income is a third lower.  While public policy has historically been used to marginalize Indigenous Americans, it can also be used as a tool to build them up.  In this paper, I examine how a tax funded Universal Basic Income (UBI) would impact Indigenous Americans.

## Background
To conduct the simulation I used data from the Census Bureau’s Annual Social and Economic Supplement (ASEC), which reports on data from 2019.  For each simulation, a UBI is given to every American of all ages and funded by a flat tax on Adjusted Gross Income (AGI).  AGI includes labor and capital income, and subtracts specific deductions like half of the self-employment taxes and contributions to certain retirement accounts.  Unlike taxable income, AGI does not account for most itemized deductions or the standard deduction.

For each UBI amount ranging from \\$0 a month to \\$1,000 a month, I calculated the change in poverty rates and income distributions for both Indigenous and non-Indigenous Americans.  The table below shows the corresponding flat tax rate on AGI for each simulated monhtly UBI amount.


In [8]:
# Import Libraries
import numpy as np
import pandas as pd
import microdf as mdf
import plotly.express as px

# Import data
person = pd.read_csv('https://github.com/ngpsu22/indigenous-peoples-day/raw/main/cps_00021.csv.gz')

# Create Demographic Columns
person.columns = person.columns.str.lower()
person['child'] = person.age < 18
person['adult'] = person.age >= 18
person['native'] = person.race == 300
person['non_native'] = person.race != 300

# Calculate total AGI
person['adjginc'].replace({99999999: 0}, inplace=True)
population = person.asecwt.sum()
person['weighted_agi'] = person.adjginc * person.asecwt
total_agi = person.weighted_agi.sum()

# Calculate AGI tax rate per dollar of UBI
fed_tax_rate_per_dollar_ubi_monthly = (population * 12) / total_agi

# Create table showing tax amounts
tax_rates = pd.DataFrame(np.arange(0,1001, 50))
tax_rates.columns = ['monthly_ubi']

def tax(monthly_ubi):
    return (monthly_ubi * fed_tax_rate_per_dollar_ubi_monthly * 100).round(1)

def tax_row(row):
    return tax(row.monthly_ubi)

tax_rates['tax_rate'] = tax_rates.apply(tax_row, axis=1)
tax_rates.columns = ['Monthly UBI', "Flat Tax Rate on AGI "] 
#tax_rates.set_index('Monthly UBI', inplace=True)
tax_rates

,Monthly UBI,Flat Tax Rate on AGI
0,0,0.0
1,50,1.6
2,100,3.3
3,150,4.9
4,200,6.6
5,250,8.2
6,300,9.8
7,350,11.5
8,400,13.1
9,450,14.8


## Results
The simulation shows that a tax-funded UBI has the potential to drastically reduce the poverty rate of Indigenous Americans.  A UBI of \\$100 per month would drop Indigenous poverty below the current non-Indigenous poverty rate.  At \\$250 per month Indigenous poverty rates fall by more than half.  A larger UBI of \\$1,000 per month would drive Indigenous poverty to less than 1 percent.

Beyond just reducing the overall poverty rate a UBI could shrink or eliminate the poverty gap between Indigenous and non-Indigenous Americans.  At all rates of \\$250 per month or higher, the gap closes within 1 percentage point.  At all rates of \\$600 per month or higher, Indigenous poverty is lower than non-Indigenous poverty.

In [2]:
def ubi(race, monthly_ubi):
    """caluclate the poverty rate, median resources, mean resources, and percent of people better off for Indigenous and non-Indegenous Americans at a given UBI level.
  
    Arguments
    race: a person's race, for this simulation, either Indigenous or non-Indigenous.
    monthly_ubi: the monthly cash transfer given to each person
  
    Returns
    The poverty rate for the selected group.
    The median resources per person for the selected group.
    The mean resources per person for the selected group.
    The percent of people better off under the program for the selected group.
    """    

  
    # Create a copy of the person DataFrame
    target_persons = person.copy(deep=True)
    
    # Calculate a person's tax increase
    target_persons['tax_increase'] = (
        fed_tax_rate_per_dollar_ubi_monthly * monthly_ubi * target_persons.adjginc)
    
    # Calculate the total UBI per SPM unit.
    target_persons['total_ubi'] = (
        target_persons.spmnpers * 12 * monthly_ubi)
  
    # Calculate the total tax increase of an SPM unit
    spmu = target_persons.groupby(['spmfamunit'])[['tax_increase']].sum()
    spmu.columns = ['total_tax_increase']
    target_persons = target_persons.merge(spmu,left_on=['spmfamunit'],
                                          right_index=True)
    
    # Calculate each SPM unit's tax rate person
    target_persons['new_spm_resources'] = (target_persons.spmtotres
                                         + target_persons.total_ubi
                                         - target_persons.total_tax_increase)
    
    # Calculate the new resources per person of each SPM unit
    target_persons['new_resources_per_person'] = (target_persons.new_spm_resources
                                                / target_persons.spmnpers)
      
    # Slice the data based on Race input
    if race == 'native':
        target_persons = target_persons[target_persons.native]
    if race == 'non_native':
        target_persons = target_persons[target_persons.non_native] 
  
    # Calculate the change in poverty rate
    target_persons['poor'] = (target_persons.new_spm_resources 
                            < target_persons.spmthresh)
    total_poor = (target_persons.poor * target_persons.asecwt).sum()
    target_pop = target_persons.asecwt.sum()
    
    # Calculate percent better off
    target_persons['better_off'] = (target_persons.new_spm_resources > 
                                target_persons.spmtotres)
    total_better_off = (target_persons.better_off * target_persons.asecwt).sum()
    percent = total_better_off / target_pop * 100

    return pd.Series([mdf.weighted_median(target_persons, 'new_resources_per_person',
                                          'asecwt').round(0), mdf.weighted_mean(target_persons,
                                            'new_resources_per_person', 'asecwt').round(0), 
                      (total_poor / target_pop * 100).round(1), percent])

def ubi_row(row):  
    """ run the ubi_pov function across the rows of a DataFrame.
  
    Arguments
    row: the row of the DataFrame containing a person's race and the monthly UBI amount
  
    Returns
    The poverty rate for the selected row.
    The median resources per person for the selected row.
    The mean resources per person for the selected row.
    The percent of people better off under the program for the selected row.
    """  
    return ubi(row.race, row.monthly_ubi)

# Create a DataFrame that has each the each monthly UBI amount for each race input
summary = mdf.cartesian_product({'monthly_ubi': np.arange(0,1001,50),
                       'race': ['native', 'non_native']})

# Calculate the poverty rate for each row of the summary DataFrame
summary[['med_resources_per_person', 'mean_resources_per_person', 'poverty_rate', 'better_off']] = summary.apply(ubi_row, axis=1)

# Format text
center = {"med_resources_per_person": "Median Resources",
                "mean_resources_per_person": "Mean Resources"}
race = {"native": "Indigenous",
            "non_native": "non-Indigenous"}

summary["race"] =  summary["race"].map(race)

def line_graph(df, x, y, color, title, xaxis_title, yaxis_title):
    """Style for line graphs.
    
    Arguments
    df: DataFrame with data to be plotted.
    x: The string representing the column in df that holds the new spending in billions.
    y: The string representing the column in df that holds the poverty rate.
    color: The string representing the UBI type.
    xaxis_title: The string represnting the xaxis-title.
    yaxis_title: The string representing the yaxis-title.
    
    Returns
    Nothing. Shows the plot.
    """
    fig = px.line(df, x=x, y=y, color=color)
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        yaxis_ticksuffix='%',
        font=dict(family='Roboto'),
        hovermode='x', 
        xaxis_tickprefix='$',
        xaxis_ticksuffix='',
        plot_bgcolor='white',
        legend_title_text=''   
    )
    fig.update_traces(mode='markers+lines', hovertemplate=None)

    fig.show()

line_graph(df=summary, x='monthly_ubi', 
          y='poverty_rate', color='race',
          title= 'The impact of a UBI on Indigenous and non-Indigenous poverty',
          xaxis_title='Monthly UBI',
          yaxis_title='SPM poverty rate')

KeyboardInterrupt: 

A UBI funded by a flat tax would not just benefit those currently living in poverty; our simulation finds that 83 percent of Indigenous Americans would be better off under the program, regardless of the amount.  

Further, a UBI could reduce both the median and mean income gaps between Indigenous and non-Indigenous Americans.  A \\$500 per month UBI shrinks the gap in both median and mean resources by about a third.


In [ ]:
# Add commas
summary["med_resources_per_person"] = summary["med_resources_per_person"].astype(int).apply("{:,}".format)
summary["mean_resources_per_person"] = summary["mean_resources_per_person"].astype(int).apply("{:,}".format)

# Create identifier columns
summary = pd.melt(summary,
        id_vars=["monthly_ubi", "race"],
        value_vars=['med_resources_per_person',
                    'mean_resources_per_person'],
        var_name="resource",             
        value_name="y")

summary["resource"] = summary["resource"].map(center)

# Plot
fig = px.bar(summary, 
       x='resource',
       y="y",
       color="race",
       barmode='group',
       animation_frame='monthly_ubi',
       text='y',
       height=900, width=800,
       labels={"race": "Race",
               "monthly_ubi": "Monthly UBI",
               "y": "Resource Per Person"
             },
    color_discrete_map={'Indigenous': '#3333FF',
                        'non-Indigenous': '#CC0000',},
       title="Tax funded UBI and median/mean resources per person",
       range_y=[0, 32_000]
    )
fig.update_traces(texttemplate='$%{text}')
fig.update_layout(title_x=0.5, 
                  xaxis_title='',
                  yaxis_tickprefix='$',
                 uniformtext_minsize=9)
fig.show()

## Conclusion
Disparities between Indigenous and non-Indigenous Americans are not limited to income: inequities persist in [education](https://education.wsu.edu/documents/2015/08/native-american-achievement-gap-report.pdf/), [health outcomes](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2567901/), [incarceration rates](https://www.bjs.gov/content/pub/pdf/aic.pdf), and [life expectancy](http://ssrc-static.s3.amazonaws.com/wp-content/uploads/2015/04/Geographies-of-Opportunity-4.22.2015.pdf).  However, evidence suggests that cash transfers could help close these gaps as well.  One [study](https://www.aeaweb.org/articles?id=10.1257/app.2.1.86) followed a group of Cherokee Indian families that received unconditional cash transfers of approximately \\$4,000 annually as part of a profit sharing agreement from a local casino. When recipients of the dividend were compared to residents nearby that did not receive it, researchers found that an additional \\$4,000 per year for the poorest households increased educational attainment of children by one year by age 21, and reduced the chance of committing a minor crime by 22 percent for 16 and 17 year olds.  They also documented a decline in teenage pregnancy and substance abuse.  Other studies found cash transfers reduce [child obesity](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3380033), improve [health outcomes](http://eprints.lse.ac.uk/58059/1/__lse.ac.uk_storage_LIBRARY_Secondary_libfile_shared_repository_Content_STICERD_PEP%20discussion%20papers_pep01.pdf), and increase [life expectancy](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5510957/).  

In short, a UBI has the potential to radically change the life of the average Indigenous American for the better.  As Americans across the country take today to reflect on the history of brutal treatment of our country’s Native people, we should also consider the harm that is still being done.  Advocates and allies for Indigenous rights should see a UBI as a valuable tool to increase the overall wellbeing of Indigenous Americans and to begin to reduce some of the disparities in outcomes between Indigenous and non-Indigenous Americans.
